# Group By

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/25 08:32:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df_green = spark.read.parquet('data/pq/green/*/*')

## Green

In [8]:
df_green.registerTempTable('green')

In [17]:
df_green_hourly_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [18]:
df_green_hourly_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000002|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [26]:
df_green_hourly_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

## Yellow

In [21]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [24]:
df_yellow_hourly_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [25]:
df_yellow_hourly_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

# Joins

In [37]:
df_green_hourly_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_hourly_revenue = spark.read.parquet('data/report/revenue/yellow')

In [38]:
df_green_hourly_revenue_tmp = df_green_hourly_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_hourly_revenue_tmp = df_yellow_hourly_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [39]:
df_join = df_green_hourly_revenue_tmp.join(df_yellow_hourly_revenue_tmp, on=['hour', 'zone'], how='outer')

In [40]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [30]:
df_join.show()

+-------------------+----+------------------+--------------+------------------+--------------+
|               hour|zone|            amount|number_records|            amount|number_records|
+-------------------+----+------------------+--------------+------------------+--------------+
|2020-01-01 00:00:00|   3|              null|          null|              25.0|             1|
|2020-01-01 00:00:00|   4|              null|          null|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7| 769.7299999999996|            45| 455.1700000000001|            38|
|2020-01-01 00:00:00|  12|              null|          null|             107.0|             6|
|2020-01-01 00:00:00|  13|              null|          null|1214.8000000000002|            56|
|2020-01-01 00:00:00|  18|               7.8|             1|               5.8|             1|
|2020-01-01 00:00:00|  29|              61.3|             1|              null|          null|
|2020-01-01 00:00:00|  36|295.34000000000003|     

In [41]:
df_join = spark.read.parquet('data/report/revenue/total')

In [42]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  10|              null|                null|             42.41|                    2|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  22|              15.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000002|                   45|
|2020-01-01 00:00:00|  25| 5

In [52]:
df = spark.read.csv('taxi_zone_lookup.csv', header=True)

In [53]:
df.write.parquet('zones', mode='overwrite')

In [54]:
df_zones = spark.read.parquet('zones/')

In [55]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [56]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [60]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')